In [1]:
import sys
!{sys.executable} -m pip install -r requirements.txt



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import nltk
import spacy
import benepar
import stanza

nltk.download('wordnet', quiet=False)
nltk.download('verbnet', quiet=False)

spacy.load('en_core_web_trf')
benepar.download('benepar_en3')
stanza.download('en', logging_level='INFO')


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Nassim\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package verbnet to
[nltk_data]     C:\Users\Nassim\AppData\Roaming\nltk_data...
[nltk_data]   Package verbnet is already up-to-date!
c:\Users\Nassim\anaconda3\Lib\site-packages\thinc\shims\pytorch.py:253: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by th

In [ ]:
import pandas as pd
import spacy
import benepar
from tqdm import tqdm
import utils

tqdm.pandas()

# Pseudocode Step 1: Load spaCy language model
nlp = spacy.load('en_core_web_sm')
nlp.add_pipe('benepar', config={'model': 'benepar_en3'})


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Nassim\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package benepar_en3 to
[nltk_data]     C:\Users\Nassim\AppData\Roaming\nltk_data...
[nltk_data]   Package benepar_en3 is already up-to-date!
2025-11-02 12:55:10 INFO: Downloading default packages for language: en (English)...
2025-11-02 12:55:10 INFO: File exists: C:\Users\Nassim\stanza_resources\en\default.zip.
2025-11-02 12:55:12 INFO: Finished downloading models and saved to C:\Users\Nassim\stanza_resources.
[nltk_data] Downloading package verbnet to
[nltk_data]     C:\Users\Nassim\AppData\Roaming\nltk_data...
[nltk_data]   Package verbnet is already up-to-date!
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use th

In [ ]:
df = pd.read_csv('Anaphoric.csv')

# Identify the correct column for requirements
requirements_column = 'Raw Requirements'
if requirements_column not in df.columns:
    requirements_column = df.columns[0]
    print(f"Column 'Requirement' not found. Using the first column: '{requirements_column}'")

print(f"Dataset loaded. Initial shape: {df.shape}")

# Drop rows where the requirement text is missing (NaN)
df.dropna(subset=[requirements_column], inplace=True)

# Ensure all data in the column is of string type to prevent errors
df[requirements_column] = df[requirements_column].astype(str)

print(f"Cleaned dataset. Shape after removing empty rows: {df.shape}")
display(df.head())

# Pseudocode Step 2: Apply NLP to Data
df['Context_doc'] = df[requirements_column].progress_apply(lambda text: utils.applynlp(text, nlp))


Dataset loaded. Initial shape: (159, 10)
Cleaned dataset. Shape after removing empty rows: (159, 10)


,Domain,Raw Requirements,Manual Evaluation,Automated,ChatGPT,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,
0,Game,The Game must be programmed using the C# langu...,UA,A,A,NaN,NaN,NaN,NaN,NaN
1,Game,The User shall be able to the select username ...,UA,UA,UA,NaN,NaN,NaN,NaN,NaN
2,Game,Username and Password must be available to ent...,A,UA,UA,NaN,NaN,NaN,NaN,NaN
3,Game,The website shall enable the players to select...,A,UA,UA,NaN,NaN,NaN,NaN,NaN
4,Game,"Each team players will have Names, Contacts nu...",UA,UA,UA,NaN,NaN,NaN,NaN,NaN


 30%|███       | 48/159 [00:01<00:03, 31.74it/s]

As we are working on web-based application with backend in python, so there would be different libraries imported for python and Web App.

Each user will have their own
User Name
Email



 35%|███▌      | 56/159 [00:01<00:03, 29.84it/s]

Tasker can sent video to admin pages after confirmation, it will be added on home page.



100%|██████████| 159/159 [00:05<00:00, 28.71it/s]


In [ ]:
# Pseudocode Step 3: Define Pronouns
pronouns_list = [
    "i", "me", "my", "mine", "myself", "you", "your", "yours", "yourself",
    "he", "him", "his", "himself", "she", "her", "hers", "herself",
    "it", "its", "itself", "we", "us", "our", "ours", "ourselves",
    "they", "them", "their", "theirs", "themselves"
]

# Pseudocode Step 4: Initialize Data Structures
processed_triples = []
ids_used = set()  # Use a set for fast checking of uniqueness

print("Generating (Pronoun, Candidate Antecedent) pairs...")

# Pseudocode Step 6: For each unique context, find pronouns and antecedents
for i, row in tqdm(df.iterrows(), total=df.shape[0], desc="Generating Triples"):
    doc = row['Context_doc']
    
    if not isinstance(doc, spacy.tokens.Doc):
        continue

    pronouns_in_doc = utils.findPronouns(doc, pronouns_list)
    
    for j, pronoun in enumerate(pronouns_in_doc):
        # Generate a unique ID for each pronoun-context pair
        base_id = f"{i}-{pronoun.text.lower()}-{j}"
        
        # Ensure ID is unique
        k = 0
        pronoun_id = base_id
        while pronoun_id in ids_used:
            k += 1
            pronoun_id = f"{base_id}-{k}"
        ids_used.add(pronoun_id)
        
        candidate_antecedents = utils.getNPs(doc, pronoun)
        
        for candidate in candidate_antecedents:
            # Append data to list
            processed_triples.append([
                pronoun_id,
                doc,
                pronoun,
                pronoun.i,
                candidate,
                row['Manual Evaluation']
            ])

# Pseudocode Step 7: Create DataFrame from Results
triples_df = pd.DataFrame(processed_triples, columns=[
    "Id", "Context", "Pronoun", "Position", 
    "Candidate_Antecedent", "Manual Evaluation"
])

print(f"Generated {len(triples_df)} pronoun-antecedent pairs.")
display(triples_df.head())

Generating (Pronoun, Candidate Antecedent) pairs...


Generating Triples: 100%|██████████| 159/159 [00:00<00:00, 14992.45it/s]

\nGenerated 461 pronoun-antecedent pairs.


,Id,Context,Pronoun,Position,Candidate_Antecedent,Manual Evaluation
0,0-its-0,"(The, Game, must, be, programmed, using, the, ...",its,11,"(The, Game)",UA
1,0-its-0,"(The, Game, must, be, programmed, using, the, ...",its,11,"(the, C, #, language)",UA
2,0-its-0,"(The, Game, must, be, programmed, using, the, ...",its,11,"(the, C, #, language, and, its, libraries)",UA
3,1-their-0,"(The, User, shall, be, able, to, the, select, ...",their,12,"(The, User)",UA
4,1-their-0,"(The, User, shall, be, able, to, the, select, ...",their,12,"(the, select, username)",UA


In [ ]:
# Make a copy of the original triples DataFrame to avoid modifying it directly
final_df = triples_df.copy()

# Create a dictionary to store the modified context with hashed pronouns
hashed_context_map = {}

# Group the DataFrame by pronoun ID to process each pronoun instance separately
for pronoun_id, group in final_df.groupby('Id'):
    # Take the first row of the group (all rows share the same context and pronoun)
    first_row = group.iloc[0]
    pronoun_token = first_row['Pronoun']
    context_doc = first_row['Context']
    
    # Create a hashed version of the pronoun using its text and index
    hashed_pronoun = f"{pronoun_token.text}#{pronoun_id.split('-')[-1]}"

    # Replace the original pronoun in the context with the hashed version
    hashed_context_map[pronoun_id] = (
        context_doc[:pronoun_token.i].text_with_ws + 
        hashed_pronoun + 
        context_doc[pronoun_token.i + 1:].text_with_ws
    )

# Add the new hashed context to the DataFrame
final_df['Hashed_Context'] = final_df['Id'].map(hashed_context_map)

# Convert spaCy token and span objects to plain text for serialization
final_df['Pronoun'] = final_df['Pronoun'].apply(lambda token: token.text)
final_df['Candidate_Antecedent'] = final_df['Candidate_Antecedent'].apply(lambda span: span.text)

# Remove the original 'Context' column since it's no longer needed
final_df.drop(columns=['Context'], inplace=True)

# Reorder and select the final columns for SpanBERT input
final_df = final_df[[
    'Id', 'Hashed_Context', 'Pronoun', 'Position', 
    'Candidate_Antecedent', 'Manual Evaluation'
]]

# Save file
output_filename = 'anaphoric_ambiguity_spanbert_input.csv'
final_df.to_csv(output_filename, index=False)

display(final_df.head())

Finalizing DataFrame: Creating Hashed Context and serializing data...
\nPreprocessing complete. Data ready for SpanBERT and saved to 'anaphoric_ambiguity_spanbert_input.csv'


,Id,Hashed_Context,Pronoun,Position,Candidate_Antecedent,Manual Evaluation
0,0-its-0,The Game must be programmed using the C# langu...,its,11,The Game,UA
1,0-its-0,The Game must be programmed using the C# langu...,its,11,the C# language,UA
2,0-its-0,The Game must be programmed using the C# langu...,its,11,the C# language and its libraries,UA
3,1-their-0,The User shall be able to the select username ...,their,12,The User,UA
4,1-their-0,The User shall be able to the select username ...,their,12,the select username,UA
